In [1]:
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras import layers, models
from tensorflow.keras.models import Model
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score



In [3]:
# Load and preprocess the MNIST dataset
(x_train,y_train),(x_test,y_test) = mnist.load_data()

x_train = np.stack([x_train]*3,axis=-1)
x_test = np.stack([x_test]*3,axis=-1)

x_train = np.array([np.pad(img, ((10, 10), (10, 10), (0, 0)), mode='constant') for img in x_train])
x_test = np.array([np.pad(img, ((10, 10), (10, 10), (0, 0)), mode='constant') for img in x_test])

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

x_train = preprocess_input(x_train)
x_test = preprocess_input(x_test)


In [5]:
# Load the pre-trained VGG16 model
input_layer = layers.Input(shape=(48, 48, 3))
model_vgg16 = VGG16(weights='imagenet', input_tensor=input_layer, include_top=False)

for layer in model_vgg16.layers:
    layer.trainable = False
    
last_layer = model_vgg16.output

flatten = layers.Flatten()(last_layer)

dense1 = layers.Dense(64, activation='relu')(flatten)
dense2 = layers.Dense(32, activation='relu')(dense1)

output_layer = layers.Dense(10, activation='softmax')(dense2)

model = models.Model(inputs=input_layer, outputs=output_layer)

model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 48, 48, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 48, 48, 64)     │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 48, 48, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 24, 24, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 24, 24, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 24, 24, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 12, 12, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 12, 12, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 12, 12, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 12, 12, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 6, 6, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 6, 6, 512)      │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 6, 6, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 6, 6, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 3, 3, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 3, 3, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 3, 3, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 3, 3, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 1, 1, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │        32,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │           330 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,749,930 (56.27 MB)

 Trainable params: 35,242 (137.66 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [7]:
# Extract features using the VGG16 model
feature_extractor = Model(inputs=model_vgg16.input, outputs=flatten)


In [11]:
features_train = feature_extractor.predict(x_train)
features_test = feature_extractor.predict(x_test)

1875/1875 ━━━━━━━━━━━━━━━━━━━━ 104s 55ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 16s 52ms/step


2024-06-24 14:59:04.437329: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] PluggableGraphOptimizer failed: INVALID_ARGUMENT: Failed to deserialize the `graph_buf`.


In [13]:
# Train a single dense layer (ANN model) on the extracted features
ann_input = layers.Input(shape=(features_train.shape[1],))

dense_layer = layers.Dense(64, activation='relu')(ann_input)

output_layer = layers.Dense(10, activation='softmax')(dense_layer)

ann_model = models.Model(inputs=ann_input, outputs=output_layer)
ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
ann_model.fit(features_train, y_train, epochs=10, batch_size=32, validation_data=(features_test, y_test))

Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 17s 7ms/step - accuracy: 0.2501 - loss: 2.3364 - val_accuracy: 0.3898 - val_loss: 1.5842
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 12s 6ms/step - accuracy: 0.4386 - loss: 1.5155 - val_accuracy: 0.5207 - val_loss: 1.3738
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 13s 7ms/step - accuracy: 0.5037 - loss: 1.3744 - val_accuracy: 0.5150 - val_loss: 1.2942
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 16s 9ms/step - accuracy: 0.5490 - loss: 1.2726 - val_accuracy: 0.5930 - val_loss: 1.1566
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 13s 7ms/step - accuracy: 0.5907 - loss: 1.1828 - val_accuracy: 0.6198 - val_loss: 1.1377
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 13s 7ms/step - accuracy: 0.6187 - loss: 1.1110 - val_accuracy: 0.6813 - val_loss: 1.0168
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 13s 7ms/step - accuracy: 0.6441 - loss: 1.0485 - val_accuracy: 0.6860 - val_loss: 0.9820
Epoch 8/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 14s 7ms/step - accuracy: 0.6592 - loss: 1

In [15]:
from sklearn.decomposition import PCA
pca = PCA(n_components=50)  # Reduce to 50 dimensions
features_train_pca = pca.fit_transform(features_train)
features_test_pca = pca.transform(features_test)

In [17]:
# Function to train and evaluate a classifier multiple times and return the accuracies
def evaluate_classifier(classifier, features_train, y_train, features_test, y_test, n_iterations=10):
    accuracies = []
    for _ in range(n_iterations):
        classifier.fit(features_train, y_train)
        y_pred = classifier.predict(features_test)
        accuracy = accuracy_score(y_test, y_pred)
        accuracies.append(accuracy)
    return accuracies

In [21]:
# Evaluate Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_accuracies = evaluate_classifier(rf_classifier, features_train_pca, y_train, features_test_pca, y_test)
rf_mean_accuracy = np.mean(rf_accuracies)
rf_std_accuracy = np.std(rf_accuracies)
print(f"Random Forest - Mean Accuracy: {rf_mean_accuracy * 100:.2f}%, Std Dev: {rf_std_accuracy * 100:.2f}%")

Random Forest - Mean Accuracy: 89.43%, Std Dev: 0.00%
